In [2]:
import json
import pandas as pd
import numpy as np
import plotly.express as px
from jupyter_dash import JupyterDash
from dash import Dash, dcc, html, Input, Output

Import the geo.json file of European countries

In [3]:
#geojson file
data = json.load(open('countries.geojson','r'))

#reading drug data
can_dat = pd.read_excel('data/cannabis_2022.xlsx')
coc_dat = pd.read_excel('data/cocaine_2022.xlsx')
alc_dat = pd.read_excel('data/alcohol_2022.xlsx')
ecs_dat = pd.read_excel('data/ecstasy_2022.xlsx')
tob_dat = pd.read_excel('data/tobacco_2022.xlsx')
any_dat = pd.read_excel('data/any_illicit_2022.xlsx')
amp_dat = pd.read_excel('data/amphetamines_2022.xlsx')

#iso code to country name (needed to relate geojson file to drug data)
iso_trans = pd.read_html('https://en.wikipedia.org/wiki/ISO_3166-1_alpha-2#Decoding_table')[2]

#a list of all 27 countries in the European Union
table = pd.read_html('https://worldpopulationreview.com/country-rankings/european-union-countries')[0]
list_eu = [item for sublist in table.values for item in sublist]

Add country name to geojson file and keep only countries in the European Union

In [4]:
#fixing iso_trans
iso_trans = iso_trans.rename(columns = {'Country name (using title case)': 'Name'})
iso_trans['Code'] = iso_trans['Code'].apply(lambda x:str(x).lower())

#changing 'Czech Republic' for 'Czechia' as the geojson file and EU data use the latter name
list_eu[list_eu.index('Czech Republic')] = 'Czechia'

#adding name and filtering for European Union
eu_countries = []
added_countries = []

for country in data['features']:
    if len(iso_trans[iso_trans['Code'] == country['properties']['cca2']]) != 0:
        name = iso_trans[iso_trans['Code'] == country['properties']['cca2']].values[:,1][0]
        country['properties']['id'] = name
        country['id'] = name
    
    if country['properties'].get('id',0) in list_eu:
        eu_countries.append(country)
        added_countries.append(country['properties']['id']) 


Filtering drug datasets to only keep the 27 EU countries

In [5]:
df_list = [can_dat,coc_dat,alc_dat,ecs_dat,tob_dat,any_dat,amp_dat]
eu_df_list =[]
for df in df_list:
    eu_df_list.append(df[df['Country'].apply(lambda x:x in list_eu)])

Merging drug datasets and renaming columns for usability in dash app

In [6]:
# delete unused columns and change NAs to 'Not reported' for sample size
for i in np.arange(0,len(eu_df_list)):
    eu_df_list[i] = eu_df_list[i].drop(['Males',"Females"],axis = 1)
    eu_df_list[i]['Sample size'] = eu_df_list[i]['Sample size'].fillna('Not reported')

#give columns unique names per drug
prefix = ['can_','coc_','alc_','ecs_','tob_','any_','amp_']
for i in np.arange(0,len(eu_df_list)):
    eu_df_list[i].columns = pd.Series(eu_df_list[i].columns).apply(lambda x: prefix[i]+str(x) if x != 'Country' else x)

# merge dataframes
drug_data = pd.DataFrame({'Country':pd.Series(list_eu)})
for df in eu_df_list:
    drug_data = pd.merge(drug_data,df)


In [7]:
drug_data

,Country,can_Year,can_Sample size,can_Total,coc_Year,coc_Sample size,coc_Total,alc_Year,alc_Sample size,alc_Total,...,ecs_Total,tob_Year,tob_Sample size,tob_Total,any_Year,any_Sample size,any_Total,amp_Year,amp_Sample size,amp_Total
0,Austria,2020,4650.0,3.6,2020.0,4650.0,0.3,2020.0,4650.0,77.2,...,0.1,2020.0,4650.0,26.1,2020.0,4650.0,5.1,2015.0,3477.0,0.1
1,France,2017,20665.0,6.4,NaN,Not reported,NaN,NaN,Not reported,NaN,...,NaN,NaN,Not reported,NaN,NaN,Not reported,NaN,NaN,Not reported,NaN
2,Malta,2013,Not reported,0.4,NaN,Not reported,NaN,2013.0,Not reported,58.8,...,NaN,2013.0,Not reported,27.4,NaN,Not reported,NaN,NaN,Not reported,NaN
3,Belgium,2018,3954.0,4.3,NaN,Not reported,NaN,NaN,Not reported,NaN,...,NaN,2008.0,6792.0,27.7,2018.0,3954.0,5.5,NaN,Not reported,NaN
4,Germany,2018,9267.0,3.0,2018.0,9267.0,0.3,2018.0,9267.0,71.0,...,0.3,2018.0,9267.0,23.3,2018.0,9267.0,3.3,2018.0,9267.0,0.5
5,Netherlands,2020,5312.0,6.3,2020.0,5312.0,0.7,2009.0,5769.0,75.7,...,1.0,NaN,Not reported,NaN,2020.0,5312.0,7.5,2019.0,6127.0,0.8
6,Bulgaria,2020,3838.0,1.0,2020.0,3838.0,0.2,2020.0,3838.0,63.7,...,0.1,2020.0,3838.0,49.1,2020.0,3838.0,1.4,2016.0,3996.0,0.3
7,Greece,2015,1519.0,1.3,2015.0,1519.0,0.2,NaN,Not reported,NaN,...,0.1,2015.0,1519.0,41.1,2015.0,1519.0,1.3,NaN,Not reported,NaN
8,Poland,2018,3013.0,2.0,2018.0,3013.0,0.2,NaN,Not reported,NaN,...,0.3,2018.0,3013.0,34.0,2018.0,3013.0,3.0,2018.0,3013.0,0.5
9,Croatia,2019,4994.0,5.6,2019.0,4994.0,0.9,2019.0,4994.0,62.9,...,0.4,2019.0,4994.0,41.5,2019.0,4994.0,6.3,2019.0,4994.0,0.9


Creating the Dash app

In [8]:
app = JupyterDash(__name__)

app.layout = html.Div([
    html.H1('Drug consumption in the European Union',style = {'font-family':'Helvetica'}),
    html.Div('Month prevalence per European Union country for different substances (Hover for more information).'),
    html.Br(),
    html.Div(['Select an illicit substance:'
        ,dcc.Dropdown(['Cannabis','Cocaine','Alcohol','Ecstasy','Tobacco','Amphetamines','Any illicit drugs'],
                          'Cannabis', id = 'drug_input')],    
            style = {'width':'200px','height':'70px'}),
    html.Div([dcc.Graph(id = 'the graph')]),
    html.Div('*All data extracted from the 2022 annual Statistical Bulleting published by the European Monitoring Center for Drugs and Drug Addiction.'),
    html.Div('**Countries in gray: no data or not part of the European')
],style = {'font-family':'roboto'})

@app.callback(
    Output(component_id = 'the graph', component_property = 'figure'),
    Input(component_id = 'drug_input', component_property = 'value')
)
def deploy_figure(drug_selected):
    #translating dropdown selection to appropriate values
    color_dic = {'Cannabis':'can_Total','Cocaine':'coc_Total','Alcohol':'alc_Total','Ecstasy':'ecs_Total',
                'Tobacco':'tob_Total','Any illicit drugs':'any_Total','Amphetamines':'amp_Total'}
    color = color_dic[drug_selected]
    year_dic = {'Cannabis':'can_Year','Cocaine':'coc_Year','Alcohol':'alc_Year','Ecstasy':'ecs_Year','Tobacco':'tob_Year',
               'Any illicit drugs': 'any_Year','Amphetamines':'amp_Year'}
    year = year_dic[drug_selected]
    sample_dic = {'Cannabis':'can_Sample size','Cocaine':'coc_Sample size','Alcohol':'alc_Sample size',
                  'Ecstasy': 'ecs_Sample size','Tobacco':'tob_Sample size','Any illicit drugs':'any_Sample size',
                 'Amphetamines': 'amp_Sample size'}
    sample = sample_dic[drug_selected]
    fig = px.choropleth(drug_data,locations = 'Country', geojson = eu_countries, 
                   color = color, scope = 'europe',
                   color_continuous_scale = 'OrRd',
                   locationmode = 'country names',
                   hover_name='Country',
                   hover_data = [year,sample, color],
                   labels = {color: 'Last month prevalence (%)', year: 'Year of survey', sample: 'Sample size'})
    fig.update_geos(fitbounds = 'locations', landcolor = 'lightgray')
    fig.update_layout(margin={"r":100,"t":0,"l":100,"b":0})
    return fig
  
app.run_server(port = 2233, debug=False)

 * Running on http://127.0.0.1:2233/ (Press CTRL+C to quit)
127.0.0.1 - - [03/Nov/2022 17:37:14] "GET /_alive_011ed8b3-8eb3-4fe1-b984-ea95b974c5ee HTTP/1.1" 200 -


Dash app running on http://127.0.0.1:2233/


127.0.0.1 - - [03/Nov/2022 17:37:16] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [03/Nov/2022 17:37:16] "GET /assets/style.css?m=1666457448.0 HTTP/1.1" 200 -
127.0.0.1 - - [03/Nov/2022 17:37:16] "GET /_dash-component-suites/dash/deps/polyfill@7.v2_6_2m1666437973.12.1.min.js HTTP/1.1" 200 -
127.0.0.1 - - [03/Nov/2022 17:37:16] "GET /_dash-component-suites/dash/deps/react@16.v2_6_2m1666437973.14.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [03/Nov/2022 17:37:16] "GET /_dash-component-suites/dash/deps/react-dom@16.v2_6_2m1666437973.14.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [03/Nov/2022 17:37:16] "GET /_dash-component-suites/dash/deps/prop-types@15.v2_6_2m1666437973.8.1.min.js HTTP/1.1" 200 -
127.0.0.1 - - [03/Nov/2022 17:37:16] "GET /_dash-component-suites/dash/dash-renderer/build/dash_renderer.v2_6_2m1666437973.min.js HTTP/1.1" 200 -
127.0.0.1 - - [03/Nov/2022 17:37:16] "GET /_dash-component-suites/dash/dcc/dash_core_components.v2_6_2m1666437973.js HTTP/1.1" 200 -
127.0.0.1 - - [03/Nov/2022 17:37:16]

**REFERENCES** 

geojson file: https://github.com/Stefie/geojson-world

drug data: https://www.emcdda.europa.eu/data/stats2022/gps_en
